In [152]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Sh'ift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [153]:
df=pd.read_csv('../input/graduates-admission-prediction/admission_data.csv')

In [154]:
df.head()

In [155]:
df.columns

In [156]:
df.isnull().any()

In [157]:
df.describe()

In [158]:
df.describe()

In [159]:
df.columns

In [160]:
sns.countplot(x='Research',data=df)

In [161]:
df.plot.scatter('Chance of Admit ','SOP', color = 'darkturquoise')

In [193]:
df.plot.scatter('University Rating','GRE Score', color = 'magenta')

# BETTER THE UNIVERSITY RATING, HIGHER THE GRE SCORE REQUIRED TO SECURE AN ADMIT.
# FOR A UNI RATING OF 5 A MINIMUM OF 320+ GRE SCORE IS REQUIRED

In [196]:
df.plot.scatter('CGPA','GRE Score', color = 'darkturquoise')

In [198]:
df.plot.scatter('CGPA','TOEFL Score', color = 'orange')

# THOSE WHO MAINTAIN A CGPA OF 8.5 CGPA + WHILE IN COLLEGE TEND TO SCORE 110+ IN TOEFL


# THOSE WHO HAVE A HIGH CGPA IN COLLEGE TEND TO SCORE BETTER IN THE GRE

In [162]:
df.plot.scatter('Chance of Admit ','GRE Score', color = 'red')

#  THE CHANCE OF GETTING AN ADMIT INCREASES WITH HIGH GRE SCORE.
#  THE IDEAL GRE SCORE IS 320+

In [163]:
df.plot.scatter('Chance of Admit ','CGPA', color = 'darkturquoise')

# CHANCE OF ADMIT INCREASES WITH CGPA.
# IDEAL CGPA IS 8.5 CGPA +

In [164]:
df.plot.scatter('Chance of Admit ','TOEFL Score', color = 'blue')

# CHANCE OF ADMIT INCREASES WITH TOEFL SCORE
# IDEAL SCORE IS 110 +

In [165]:
X=df.drop(["Chance of Admit ",],axis=1)
y=df['Chance of Admit ']

In [166]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(),linewidths=0.5,annot=True)
plt.show()

In [167]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.17,random_state=0)

In [168]:
X_train

In [169]:
X_test

In [170]:
y_train

In [171]:
y_test

In [172]:
from sklearn.ensemble import RandomForestRegressor
rf_classifier=RandomForestRegressor(n_estimators=100).fit(X_train,y_train)
prediction=rf_classifier.predict(X_test)

In [173]:
from sklearn.metrics import r2_score
r2_score(y_test, prediction)

In [174]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 3000, num = 20)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,20)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,15,20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 3, 5,7,9,15]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':["squared_error", "absolute_error", "poisson"]}
print(random_grid)

In [175]:
rf=RandomForestRegressor()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,
                               random_state=100,n_jobs=-1)
### fit the randomized model
rf_randomcv.fit(X_train,y_train)

In [176]:
best_random_grid=rf_randomcv.best_estimator_

In [177]:
best_random_grid

In [178]:
y_pred=best_random_grid.predict(X_test)

In [179]:
r2_score(y_test, y_pred)

In [ ]:
print(confusion_matrix(y_test,y_pred))
print(accuracy_score(y_test,y_pred))
print(classification_report(y_test,y_pred))

In [180]:
rf_randomcv.best_params_

In [181]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'criterion': [rf_randomcv.best_params_['criterion']],
    'max_depth': [rf_randomcv.best_params_['max_depth']],
    'max_features': [rf_randomcv.best_params_['max_features']],
    'min_samples_leaf': [rf_randomcv.best_params_['min_samples_leaf'], 
                         rf_randomcv.best_params_['min_samples_leaf']+2, 
                         rf_randomcv.best_params_['min_samples_leaf'] + 4],
    'min_samples_split': [rf_randomcv.best_params_['min_samples_split'] +4,
                          rf_randomcv.best_params_['min_samples_split'] +1,
                          rf_randomcv.best_params_['min_samples_split'], 
                          rf_randomcv.best_params_['min_samples_split'] +2,
                          rf_randomcv.best_params_['min_samples_split'] + 3],
    'n_estimators': [rf_randomcv.best_params_['n_estimators'] - 200, rf_randomcv.best_params_['n_estimators'] - 100, 
                     rf_randomcv.best_params_['n_estimators'], 
                     rf_randomcv.best_params_['n_estimators'] + 100, rf_randomcv.best_params_['n_estimators'] + 200,
    rf_randomcv.best_params_['n_estimators'] + 400,rf_randomcv.best_params_['n_estimators'] + 600, rf_randomcv.best_params_['n_estimators'] + 800]
}

print(param_grid)

In [182]:
rf=RandomForestRegressor()
grid_search=GridSearchCV(estimator=rf,param_grid=param_grid,cv=10,n_jobs=-1,verbose=2)
grid_search.fit(X_train,y_train)

In [183]:
grid_search.best_estimator_

In [184]:
best_grid=grid_search.best_estimator_

In [185]:
best_grid

In [186]:
y_pred=best_grid.predict(X_test)
r2_score(y_test, y_pred)

In [187]:
best_random_grid


In [188]:
best_grid

In [189]:
# Plotting scatter plot with best fit line to check fit.
plt.scatter(y_test,y_pred)

#obtain m (slope) and b(intercept) of linear regression line
m, b = np.polyfit(y_test, y_pred, 1)

#Plotting best fit line
plt.plot(y_test, m*y_test+b, color = 'red')